<a href="https://colab.research.google.com/github/Liraken/P04-Weather-Risk/blob/main/P4_LSTM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as test_train_split
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import r2_score
from google.colab import files
import tensorflow as tf

In [2]:
uploaded = files.upload()

Saving storm_data_trunc.csv to storm_data_trunc.csv


In [3]:
# Custom callback to print R^2 value during training
class R2Callback(Callback):
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.X_train)
        r2 = r2_score(self.y_train, y_pred)
        print(f"Epoch {epoch+1}, R^2: {r2}")

In [4]:
df=pd.read_csv('storm_data_trunc.csv')
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,DURATION_SEC,REGION_NORTHEAST,REGION_NORTHERN ROCKIES AND PLAINS,REGION_NORTHWEST,...,EVENT_TYPE_Coastal Flood,EVENT_TYPE_Flash Flood,EVENT_TYPE_Flood,EVENT_TYPE_Hail,EVENT_TYPE_Hurricane,EVENT_TYPE_Hurricane (Typhoon),EVENT_TYPE_Storm Surge/Tide,EVENT_TYPE_Tornado,EVENT_TYPE_Tropical Storm,EVENT_TYPE_Wildfire
0,1994-03-27 11:32:00,5000000.0,34.43,-85.98,34.47,-85.78,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1994-05-15 13:47:00,0.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1994-03-27 15:50:00,0.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1994-03-27 15:50:00,0.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1994-03-27 15:55:00,0.0,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
df=df.drop(columns=['BEGIN_LAT','BEGIN_LON','END_LAT','END_LON','DURATION_SEC'])
df.sort_values(by=['BEGIN_DATE_TIME'],inplace=True)
df.head()


,BEGIN_DATE_TIME,DAMAGE_PROPERTY,REGION_NORTHEAST,REGION_NORTHERN ROCKIES AND PLAINS,REGION_NORTHWEST,REGION_OHIO VALLEY,REGION_SOUTH,REGION_SOUTHEAST,REGION_SOUTHWEST,REGION_UPPER MIDWEST,...,EVENT_TYPE_Coastal Flood,EVENT_TYPE_Flash Flood,EVENT_TYPE_Flood,EVENT_TYPE_Hail,EVENT_TYPE_Hurricane,EVENT_TYPE_Hurricane (Typhoon),EVENT_TYPE_Storm Surge/Tide,EVENT_TYPE_Tornado,EVENT_TYPE_Tropical Storm,EVENT_TYPE_Wildfire
6962,1994-01-03 14:45:00,500000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6963,1994-01-03 14:45:00,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7380,1994-01-03 17:05:00,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7449,1994-01-03 17:08:00,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7011,1994-01-03 17:09:00,50000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
#df['YEAR']=df['BEGIN_DATE_TIME'].dt.year
#df['MONTH']=df['BEGIN_DATE_TIME'].dt.month
#df=df.drop(columns=['BEGIN_DATE_TIME'])
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].dt.to_period('M')
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df['BEGIN_DATE_TIME'] = df['BEGIN_DATE_TIME'].map(pd.Timestamp.timestamp)
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,REGION_NORTHEAST,REGION_NORTHERN ROCKIES AND PLAINS,REGION_NORTHWEST,REGION_OHIO VALLEY,REGION_SOUTH,REGION_SOUTHEAST,REGION_SOUTHWEST,REGION_UPPER MIDWEST,...,EVENT_TYPE_Coastal Flood,EVENT_TYPE_Flash Flood,EVENT_TYPE_Flood,EVENT_TYPE_Hail,EVENT_TYPE_Hurricane,EVENT_TYPE_Hurricane (Typhoon),EVENT_TYPE_Storm Surge/Tide,EVENT_TYPE_Tornado,EVENT_TYPE_Tropical Storm,EVENT_TYPE_Wildfire
6962,757382400.0,500000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6963,757382400.0,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7380,757382400.0,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7449,757382400.0,5000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7011,757382400.0,50000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
#scaler = MinMaxScaler(feature_range=(0, 1))
#data_normalized = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)


In [15]:
X=df[['BEGIN_DATE_TIME', 'DAMAGE_PROPERTY', 'REGION_NORTHEAST',
       'REGION_NORTHERN ROCKIES AND PLAINS', 'REGION_NORTHWEST',
       'REGION_OHIO VALLEY', 'REGION_SOUTH', 'REGION_SOUTHEAST',
       'REGION_SOUTHWEST', 'REGION_UPPER MIDWEST', 'REGION_WEST']]
y=df[['DAMAGE_PROPERTY']]

In [16]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_normalized = scaler.fit_transform(X)

# Normalize the target
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_normalized = scaler_y.fit_transform(y)

In [17]:
#X_train, X_test, y_train, y_test = test_train_split(X,y,test_size=.2,shuffle=False)

In [48]:
tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(X_normalized):
    X_train, X_val = X_normalized[train_index], X_normalized[val_index]
    y_train, y_val = y_normalized[train_index], y_normalized[val_index]

In [42]:
model.reset_states()

In [43]:
model=Sequential([LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
                  Dense(1)])

In [44]:
# Reduce the learning rate by a factor of 0.1 every 10 epochs.
learning_rate = 0.1
decay_rate = 0.1
decay_steps = 10

# Create a learning rate schedule.
learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

# Create an optimizer with the learning rate schedule.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)


In [45]:
model.compile(optimizer=optimizer, loss='mse')
r2_callback = R2Callback(X_train, y_train)

In [46]:
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[r2_callback])

Epoch 1/100
12956/12956 [==============================] - 33s 3ms/step
Epoch 1, R^2: -0.04017060857276267
12956/12956 [==============================] - 111s 8ms/step - loss: 3041510618562560.0000
Epoch 2/100
12956/12956 [==============================] - 34s 3ms/step
Epoch 2, R^2: -0.04017060857276267
12956/12956 [==============================] - 113s 9ms/step - loss: 3035449345966080.0000
Epoch 3/100
12956/12956 [==============================] - 34s 3ms/step
Epoch 3, R^2: -0.04017060857276267
12956/12956 [==============================] - 115s 9ms/step - loss: 3035401564454912.0000
Epoch 4/100
12956/12956 [==============================] - 34s 3ms/step
Epoch 4, R^2: -0.04017060857276267
12956/12956 [==============================] - 112s 9ms/step - loss: 3035436461064192.0000
Epoch 5/100
12956/12956 [==============================] - 38s 3ms/step
Epoch 5, R^2: -0.04017060857276267
12956/12956 [==============================] - 117s 9ms/step - loss: 3035551619874816.0000
Epoch 6/10

KeyboardInterrupt: 